# Imports

In [1]:
import pandas as pd
import numpy as np
import random

In [ ]:
# Constantes
SEED = 42
random.seed(SEED)

# Preprocesar los datasets KT3 y KT1

In [3]:
kt3_df = pd.read_csv("./kt3_df.csv")

In [4]:
# Definir una función para determinar el valor de answered_correctly
def calculate_answered_correctly(row):
    if pd.isnull(row['user_answer']) or pd.isnull(row['correct_answer']):
        return -1
    elif row['user_answer'] == row['correct_answer']:
        return 1
    else:
        return 0

# Aplicar la función a cada fila del DataFrame df
kt3_df['answered_correctly'] = kt3_df[kt3_df["content_type"] == "q"].apply(calculate_answered_correctly, axis=1)
kt3_df = kt3_df.drop(columns=["correct_answer"])

In [5]:
kt3_df['answered_correctly'].value_counts()

answered_correctly
1.0    392463
0.0    294699
Name: count, dtype: int64

In [8]:
kt3_df['content_id'] = kt3_df['item_id'].str[1:]

In [9]:
kt1_df = pd.read_csv("./kt1_df.csv")
kt1_df

,timestamp,solving_id,question_id,user_answer,elapsed_time,user_id
0,1563797068117,1,q6525,b,22000,u10030
1,1563797093153,2,q320,b,19000,u10030
2,1563797107951,3,q5659,b,12000,u10030
3,1563797123524,4,q4842,d,13000,u10030
4,1563797136092,5,q3884,a,10000,u10030
...,...,...,...,...,...,...
941949,1562654085927,169,q1588,c,26666,u996
941950,1562654085928,169,q1589,d,26666,u996
941951,1562654173463,170,q2515,c,25333,u996
941952,1562654173503,170,q2514,a,25333,u996


In [10]:
kt1_df = kt1_df.drop(columns=['timestamp', 'solving_id'])

In [11]:
# Filtrar el DataFrame df para quedarse solo con las filas cuyos user_id estén en kt1_df
kt3_q_df_filtered = kt3_df[kt3_df['user_id'].isin(kt1_df['user_id'].unique())]

# Unir los datasets

In [12]:
kt3_q_df_filtered = kt3_q_df_filtered.rename(columns={"item_id": "question_id"})
merged_df = pd.merge(
    kt3_q_df_filtered, kt1_df, on=["user_id", "question_id", "user_answer"], how="left"
)
merged_df = merged_df.rename(columns={'question_id': 'item_id'})
merged_df

,timestamp,item_id,user_answer,user_id,content_type,bundle_id,answered_correctly,content_id,elapsed_time
0,1567396097400,q921,c,u10030,q,b921,0.0,921,NaN
1,1567396102402,q921,a,u10030,q,b921,0.0,921,20000.0
2,1567396112987,e921,NaN,u10030,e,NaN,NaN,921,NaN
3,1567396132608,q1240,c,u10030,q,b1240,0.0,1240,17000.0
4,1567396154846,q589,b,u10030,q,b589,0.0,589,19000.0
...,...,...,...,...,...,...,...,...,...
1318229,1562669558978,e5194,NaN,u996,e,NaN,NaN,5194,NaN
1318230,1562669592456,q6641,a,u996,q,b5107,1.0,6641,NaN
1318231,1562669622801,q6642,c,u996,q,b5107,1.0,6642,NaN
1318232,1562669644966,q6643,c,u996,q,b5107,0.0,6643,NaN


In [13]:
merged_df.content_type.value_counts()

content_type
q    811491
e    471295
l     35448
Name: count, dtype: int64

In [14]:
len(merged_df['user_id'].unique())

8758

In [15]:
merged_df.loc[merged_df["content_type"].isin(["l", "e"]), "answered_correctly"] = -1
merged_df.loc[merged_df["content_type"].isin(["l", "e"]), "user_answer"] = -1
merged_df.loc[merged_df["content_type"].isin(["l", "e"]), "elapsed_time"] = -1

# Crear Ednet dataset

## Seleccionar una parte del dataset

In [17]:
user_ids = merged_df['user_id'].unique()

# Seleccionar 100 usuarios al azar
selected_user_ids = random.sample(list(user_ids), 200)
df = merged_df[merged_df['user_id'].isin(selected_user_ids)]
df.shape

(30771, 9)

## Crear columnas nuevas

### `bundle_had_explanation`
(bool): Indica si el usuario vio una explicación después de responder al lote de preguntas anterior (`bundle_id`), ignorando cualquier lección entre medio. El valor se comparte a través del mismo lote de preguntas y es nulo para el primer lote de preguntas de un usuario. Generalmente, las primeras preguntas que ve un usuario forman parte de una prueba de diagnóstico inicial en la que no recibieron retroalimentación.

In [18]:
df = df.sort_values(by=["user_id", "timestamp"])

# Definimos una función para obtener el bundle anterior con una explicación
def has_seen_explanation(row, df):
    # Filtramos los registros anteriores del mismo usuario
    previous_rows = df[(df['user_id'] == row['user_id']) & 
                        (df['timestamp'] < row['timestamp'])]
    
    # Buscamos si en los registros anteriores existe una explicación ('e')
    # donde el item_id tiene el mismo número que el bundle_id actual.
    for idx, previous_row in previous_rows[::-1].iterrows():  # Iterar en orden inverso
        if previous_row['content_type'] == 'e':
            # Comprobar si el número de item_id (explicación eXXXX) coincide con bundle_id (bXXXX)
            if previous_row['item_id'][1:] == row['bundle_id'][1:]:
                return True  # Explicación encontrada
        if previous_row['content_type'] == 'q':
            break  # Detenerse si encontramos otra pregunta antes de una explicación
    return False

# Aplicamos la función a las filas de preguntas (content_type == 'q')
df['bundle_had_explanation'] = df.apply(
    lambda row: has_seen_explanation(row, df) if row['content_type'] == 'q' else None,
    axis=1
)

### `prior_question_elapsed_time` 
(float32): El tiempo promedio en milisegundos que tardó un usuario en responder cada pregunta en el lote de preguntas anterior, ignorando cualquier lección entre medio (es nulo para el primer lote de preguntas).

In [19]:
df = df.sort_values(by=["user_id", "timestamp"])

if "content_type" in df.columns:
    merged_df_questions = df[df["content_type"] == "q"].copy()
else:
    merged_df_questions = df.copy()

# Calcular el tiempo promedio por bundle para cada usuario
merged_df_questions["avg_time_per_bundle"] = merged_df_questions.groupby(
    ["user_id", "bundle_id"]
)["elapsed_time"].transform("mean")

merged_df_questions = merged_df_questions.sort_values(by=["user_id", "timestamp"])

# Obtener el tiempo promedio del bundle anterior
merged_df_questions["prior_question_elapsed_time"] = merged_df_questions.groupby(
    "user_id"
)["avg_time_per_bundle"].shift()
merged_df_questions = merged_df_questions.drop(columns=["avg_time_per_bundle"])

# Actualizar el dataframe original directamente
df["prior_question_elapsed_time"] = np.nan
df.loc[merged_df_questions.index, "prior_question_elapsed_time"] = (
    merged_df_questions["prior_question_elapsed_time"]
)
df["prior_question_elapsed_time"] = df[
    "prior_question_elapsed_time"
].astype("float32")

df = df.sort_values(by=['user_id', 'timestamp'])

### `cumulative_correct_answers`
(int): El número acumulado de respuestas correctas que ha tenido el estudiante hasta ese momento. Aporta un indicador directo de cuánto ha acertado el estudiante hasta esa interacción, lo que puede ayudar a predecir su desempeño futuro.

In [20]:
df = df.sort_values(by=["user_id", "timestamp"])

mask = df["content_type"] == "q"

df.loc[mask, "cumulative_correct_answers"] = (
    df[mask]
    .sort_values(by=["user_id", "timestamp"])
    .groupby("user_id")["answered_correctly"]
    .cumsum()
)
df.loc[~mask, "cumulative_correct_answers"] = None

### `recent_accuracy` 
(float): La precisión reciente del estudiante (en las últimas 5 preguntas), calculada como el porcentaje de respuestas correctas en un intervalo corto de tiempo. Este indicador puede ser útil para medir el "momentum" del estudiante, es decir, si está mejorando o empeorando en su desempeño.

In [21]:
df = df.sort_values(by=["user_id", "timestamp"])

mask = df["content_type"] == "q"

df = df.sort_values(by=["user_id", "timestamp"])

df.loc[mask, "recent_accuracy"] = (
    df[mask].groupby("user_id")["answered_correctly"]
    .rolling(window=5, min_periods=1)
    .mean()
    .reset_index(level=0, drop=True)
)
df.loc[~mask, "recent_accuracy"] = None


### `cumulative_explanations_seen`
(int): El número acumulado de explicaciones que ha visto el estudiante hasta ese momento. Este valor puede correlacionarse con una mejor comprensión de los contenidos si el estudiante tiende a consultar explicaciones con frecuencia.

In [22]:
df = df.sort_values(by=["user_id", "timestamp"])

df["cumulative_explanations_seen"] = (
    df[df["content_type"] == "e"].groupby("user_id").cumcount() + 1
)

df["cumulative_explanations_seen"] = (
    df.groupby("user_id")["cumulative_explanations_seen"].ffill().fillna(0).astype(int)
)

### `cumulative_lectures_seen`
(int): El número acumulado de lecciones que ha visto el estudiante hasta ese momento.
Este valor puede correlacionarse con una mejor comprensión de los contenidos si el estudiante tiende a consultar lecciones con frecuencia.

In [23]:
df = df.sort_values(by=["user_id", "timestamp"])

df["cumulative_lectures_seen"] = (
    df[df["content_type"] == "l"].groupby("user_id").cumcount() + 1
)

df["cumulative_lectures_seen"] = (
    df.groupby("user_id")["cumulative_lectures_seen"].ffill().fillna(0).astype(int)
)

### `cumulative_responses_by_bundle`
(int): El número acumulado de preguntas respondidas por cada lote para cada estudiante.  

In [24]:
df = df.sort_values(by=["user_id", "timestamp"])

questions_df = df[df["content_type"] == "q"].copy()

df["cumulative_responses_by_bundle"] = (
    df[df["content_type"] == "q"].groupby(["user_id", "bundle_id"]).cumcount() + 1
)

# Guardar Ednet dataset

In [25]:
# Filtrar cuando content_type es "e" o "l" y actualizar bundle_id
mask = df["content_type"].isin(["e", "l"])
df.loc[mask, "bundle_id"] = "b" + df.loc[mask, "content_id"].astype(str)

In [26]:
df.to_csv('./ednet_dataset.csv', index=False)